In [1]:
import argparse

from utils import (print_yaml_config,init_or_resume_from, read_yaml)
from model_training.training_utils import load_for_inference
from constants import TOKENIZER_SEPECIAL_TOKENS
from training_datasets.dataset_utils import load_sft_dataset, load_rm_dataset
from training_datasets.collators import DialogueDataCollator




def main(conf):
    print(f"\n{'==='*10} Following are the configuration for training{'==='*10}")
    print_yaml_config(conf)
    device_map = {"":0}
    special_tokens = TOKENIZER_SEPECIAL_TOKENS["llama"]
    model, tokenizer = load_for_inference(device_map,conf,special_tokens,conf.model_name,False)
    return model, tokenizer

config = {}
conf = read_yaml('./config.yaml')
config.update(conf["default"])
config.update(conf["sft_eval"])
config["name_suffix"] = ""
config["debug"] = False
config["subset"] = "sft_eval"

# Create a Namespace object for config
config_ns = argparse.Namespace(**config)

init_or_resume_from(config_ns)

/home/alikhan/miniconda3/envs/torch_p310_setup/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


============================== Initialize from adapter output/LLama-2-7b_pre_sft_warm_20/final_checkpoint


In [2]:
config_ns.model_name = "meta-llama/Llama-2-7b-hf"
config_ns.init_from_adapter = "LLama-2-7b_ours_oasst_sftonly_bs64_qlora_default"
init_or_resume_from(config_ns)
only_oasst_model, only_oasst_tokenizer = main(config_ns)

============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sftonly_bs64_qlora_default/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: 0.9
adam_beta2: 0.999
adam_epsilon: 1e-12
adapter_number: final_checkpoint
adpater_name: null
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  system_prefix: null
  use_system_prefix: false
dataset:
  alpaca:
    max_val_set: 200
    val_split: 0.05
  dolly:
    max_val_set: 300
    val_split: 0.05
  math_instruction:
    max_val_set: 200
    val_split: 0.05
  oasst_export:
    lang: en,bg,ca,cs,da,de,en,es,fr,hr,hu,it,nl,pl,pt,ro,ru,sl,sr,sv,uk
    max_val_set: null
    val_split: 0.05
  vicuna:
    max_val_set: 800
    val_split: 0.05
  webgpt:
    max_val_set: 1000
    val_split: 0.05
debu

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.26s/it]


tokenizer size 32003
Resizing embeddings to 32016


In [3]:
config_ns.model_name = "meta-llama/Llama-2-7b-hf"
config_ns.init_from_adapter = "LLama-2-7b_full_sft_4bit_bs_64"
init_or_resume_from(config_ns)
full_sft_model, full_sft_tokenizer = main(config_ns)


============================== Initialize from adapter output/LLama-2-7b_full_sft_4bit_bs_64/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: 0.9
adam_beta2: 0.999
adam_epsilon: 1e-12
adapter_number: final_checkpoint
adpater_name: null
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  system_prefix: null
  use_system_prefix: false
dataset:
  alpaca:
    max_val_set: 200
    val_split: 0.05
  dolly:
    max_val_set: 300
    val_split: 0.05
  math_instruction:
    max_val_set: 200
    val_split: 0.05
  oasst_export:
    lang: en,bg,ca,cs,da,de,en,es,fr,hr,hu,it,nl,pl,pt,ro,ru,sl,sr,sv,uk
    max_val_set: null
    val_split: 0.05
  vicuna:
    max_val_set: 800
    val_split: 0.05
  webgpt:
    max_val_set: 1000
    val_split: 0.05
debug: false
debug_set

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.13s/it]


tokenizer size 32003
Resizing embeddings to 32016


In [5]:
config_ns.model_name = "meta-llama/Llama-2-7b-hf"
config_ns.init_from_adapter = "LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default"
config_ns.adapter_number= "final_checkpoint"
init_or_resume_from(config_ns)
ep2_qlora_sft_model, ep2_qlora_sft_tokenizer = main(config_ns)

============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: 0.9
adam_beta2: 0.999
adam_epsilon: 1e-12
adapter_number: final_checkpoint
adpater_name: null
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  system_prefix: null
  use_system_prefix: false
dataset:
  alpaca:
    max_val_set: 200
    val_split: 0.05
  dolly:
    max_val_set: 300
    val_split: 0.05
  math_instruction:
    max_val_set: 200
    val_split: 0.05
  oasst_export:
    lang: en,bg,ca,cs,da,de,en,es,fr,hr,hu,it,nl,pl,pt,ro,ru,sl,sr,sv,uk
    max_val_set: null
    val_split: 0.05
  vicuna:
    max_val_set: 800
    val_split: 0.05
  webgpt:
    max_val_set: 1000
    val_s

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.80s/it]


tokenizer size 32003
Resizing embeddings to 32016


In [4]:
config_ns.model_name = "meta-llama/Llama-2-7b-hf"
config_ns.init_from_adapter = "LLama-2-7b_ours_oasst_sft_ep_3_lr_1e4_cosine_bs64_qlora_default"
config_ns.adapter_number= "final_checkpoint"
init_or_resume_from(config_ns)
ep3_qlora_sft_model, ep3_qlora_sft_tokenizer = main(config_ns)

============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sft_ep_3_lr_1e4_cosine_bs64_qlora_default/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: 0.9
adam_beta2: 0.999
adam_epsilon: 1e-12
adapter_number: final_checkpoint
adpater_name: null
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  system_prefix: null
  use_system_prefix: false
dataset:
  alpaca:
    max_val_set: 200
    val_split: 0.05
  dolly:
    max_val_set: 300
    val_split: 0.05
  math_instruction:
    max_val_set: 200
    val_split: 0.05
  oasst_export:
    lang: en,bg,ca,cs,da,de,en,es,fr,hr,hu,it,nl,pl,pt,ro,ru,sl,sr,sv,uk
    max_val_set: null
    val_split: 0.05
  vicuna:
    max_val_set: 800
    val_split: 0.05
  webgpt:
    max_val_set: 1000
    val_s

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.35s/it]


tokenizer size 32003
Resizing embeddings to 32016


In [11]:
m_t = {
       "LLama-2-7b_full_sft_4bit_bs_64":[full_sft_model,full_sft_tokenizer,0.8,0.9,50],
       "LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default":[ep2_qlora_sft_model, ep2_qlora_sft_tokenizer,0.8,0.9,40],
       "LLama-2-7b_ours_oasst_sft_only_bs64_qlora_default": [only_oasst_model, only_oasst_tokenizer,0.85,0.9,40],
       "LLama-2-7b_ours_oasst_sft_ep_3_lr_1e4_cosine_bs64_qlora_default":[ep3_qlora_sft_model, ep3_qlora_sft_tokenizer,0.8,0.9,0]
}

In [7]:
def get_pred(m_t,input):
    for key,value in m_t.items():
        model,tokenizer,temp,top_p,top_k = value
        tokens = tokenizer(input,return_tensors='pt')
        output = model.generate(**tokens,
                        max_new_tokens=512,
                        do_sample=True,
                        top_p=top_p,
                        top_k=top_k,
                        repetition_penalty=1.2,
                        temperature=temp,
                    )
        print(f"\n\nOutput for model {key}:\n{'-'*100}")
        print(tokenizer.decode(output[0], skip_special_tokens=False))




In [21]:

input = """<|prompter|>Considering the advancements in quantum computing and its potential implications for classical encryption methods, explain how current cybersecurity measures might need to adapt, and what challenges or benefits this new computational paradigm could present to the fields of data privacy and digital communication.</s><|assistant|>"""
get_pred(m_t,input)




Output for model LLama-2-7b_full_sft_4bit_bs_64:
----------------------------------------------------------------------------------------------------
<s><|prompter|> Considering the advancements in quantum computing and its potential implications for classical encryption methods, explain how current cybersecurity measures might need to adapt, and what challenges or benefits this new computational paradigm could present to the fields of data privacy and digital communication.</s><|assistant|> With the advent of quantum computers and their ability to break certain types of cryptographic algorithms used in traditional security measures such as RSA and elliptic curve, it is important that existing systems are updated with newer, more secure ones which can be protected against these attacks.
The main challenge faced by organizations will likely include creating a balance between providing adequate levels of protection while also keeping up with rapidly changing technologies.  Additionally